In [1]:
import imp
import json
import os
import pickle
import milsed
import numpy as np
import math

Using TensorFlow backend.


In [2]:
cd ~/dev/milsed/models/

/home/js7561/dev/milsed/models


In [3]:
# SETUP STUFF
OUTPUT_PATH = '/home/js7561/dev/milsed/models/resources/'
train = imp.load_source('train', '/home/js7561/dev/milsed/models/02_train.py')
pump = pickle.load(open('/home/js7561/dev/milsed/models/resources/pump.pkl', 'rb'))
with open(os.path.join(OUTPUT_PATH, 'index_test.json'), 'r') as fp:
    test_idx = json.load(fp)['id']
pumpfolder = '/beegfs/js7561/datasets/dcase2017/task4_official/combined/features_silence/'
labelfile = ('/beegfs/js7561/datasets/dcase2017/task4_official/combined/metadata/labels/'
             '/groundtruth_strong_label_testing_set.csv')
duration = 10.0

In [4]:
def create_ensemble(model_list, ensemble_version, use_orig_duration=True, save=True):
    # Compute results
    models = []
    for version in model_list:
        weight_path = os.path.join(OUTPUT_PATH, version, 'model.h5')
        params = json.load(open(os.path.join(OUTPUT_PATH, version, 'params.json'), 'r'))
        model, inputs, outputs = train.MODELS[params['modelname']](pump, alpha=1.0)
        model.load_weights(weight_path)
        models.append(model)
        
    print('Computing results:')
    results, results_wfs = milsed.eval.score_ensemble_wfs(
        OUTPUT_PATH, pump, models, test_idx, pumpfolder, labelfile, duration, ensemble_version, 
        use_tqdm=True, use_orig_duration=use_orig_duration, save_jams=save)
        
    # Save them to disk
    if save:
        resultsfile = os.path.join(OUTPUT_PATH, ensemble_version, 'results.json')
        with open(resultsfile, 'w') as fp:
            json.dump(results, fp, indent=2)
            
        resultsfile_wfs = os.path.join(OUTPUT_PATH, ensemble_version, 'predictions_weakfromstrong', 'results.json')
        with open(resultsfile_wfs, 'w') as fp:
            json.dump(results_wfs, fp, indent=2)
        
    return results, results_wfs

In [5]:
ensemble_version = 'ensemble001'
models = (['model007', 'model009', 'model011', 'model013', 'model015'])
results, results_wfs = create_ensemble(models, ensemble_version, use_orig_duration=True, save=True)

Evaluating the ensemble:   0%|          | 0/488 [00:00<?, ?it/s]

Computing standard results:


Evaluating the ensemble: 100%|██████████| 488/488 [04:26<00:00,  2.28it/s]
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision i

Computing weak from strong results:


Evaluating the ensemble: 100%|██████████| 488/488 [03:22<00:00,  2.57it/s]


# Create a whole bunch of ensembles using most successful models

## Best models (trained up to 21): 7, 9, 11, 15

In [6]:
models = (['model007', 'model009', 'model011', 'model015'])
combos = []

# All pairs
for i in range(4):
    for j in range(i+1, 4):
        combos.append([models[i], models[j]])
        
# All trios
for i in range(4):
    combos.append([models[i], models[np.mod(i+1, 4)], models[np.mod(i+2, 4)]])
    
# All four
combos.append(models)

In [7]:
combos

[['model007', 'model009'],
 ['model007', 'model011'],
 ['model007', 'model015'],
 ['model009', 'model011'],
 ['model009', 'model015'],
 ['model011', 'model015'],
 ['model007', 'model009', 'model011'],
 ['model009', 'model011', 'model015'],
 ['model011', 'model015', 'model007'],
 ['model015', 'model007', 'model009'],
 ['model007', 'model009', 'model011', 'model015']]

In [7]:
def nCr(n,r):
    f = math.factorial
    return f(n) // f(r) // f(n-r)

In [8]:
print(nCr(4, 2), nCr(4, 3), nCr(4, 4))

6 4 1


In [ ]:
start_version = 2

for i in range(11):

    if start_version < 10:
        ensemble_version = 'ensemble00{}'.format(start_version)
    else:
        ensemble_version = 'ensemble0{}'.format(start_version)
        
    print('\nCreating {}:\n{}'.format(ensemble_version, combos[i]))
    _ = create_ensemble(combos[i], ensemble_version, use_orig_duration=True, save=True)
    start_version += 1


Creating ensemble002:
['model007', 'model009']


Evaluating the ensemble:   0%|          | 0/488 [00:00<?, ?it/s]

Computing standard results:


Evaluating the ensemble: 100%|██████████| 488/488 [01:02<00:00,  8.27it/s]
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision i

Computing weak from strong results:


...

Evaluating the ensemble: 100%|██████████| 488/488 [01:22<00:00,  6.08it/s]



Creating ensemble009:
['model009', 'model011', 'model015']


Evaluating the ensemble:   0%|          | 0/488 [00:00<?, ?it/s]

Computing standard results:


Evaluating the ensemble:  13%|█▎        | 63/488 [00:59<14:35,  2.06s/it]

In [ ]:
start_version = 9

for i in range(7, 11):

    if start_version < 10:
        ensemble_version = 'ensemble00{}'.format(start_version)
    else:
        ensemble_version = 'ensemble0{}'.format(start_version)
        
    print('\nCreating {}:\n{}'.format(ensemble_version, combos[i]))
    _ = create_ensemble(combos[i], ensemble_version, use_orig_duration=True, save=True)
    start_version += 1


Creating ensemble009:
['model009', 'model011', 'model015']


Evaluating the ensemble:   0%|          | 0/488 [00:00<?, ?it/s]

Computing standard results:


Evaluating the ensemble: 100%|██████████| 488/488 [01:36<00:00,  5.26it/s]
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision i

Computing weak from strong results:


...

Computing standard results:


Evaluating the ensemble:   0%|          | 0/488 [00:00<?, ?it/s]

Computing weak from strong results:


Evaluating the ensemble: 100%|██████████| 488/488 [01:31<00:00,  5.27it/s]



Creating ensemble012:
['model007', 'model009', 'model011', 'model015']


In [8]:
start_version = 12

for i in range(10, 11):

    if start_version < 10:
        ensemble_version = 'ensemble00{}'.format(start_version)
    else:
        ensemble_version = 'ensemble0{}'.format(start_version)
        
    print('\nCreating {}:\n{}'.format(ensemble_version, combos[i]))
    _ = create_ensemble(combos[i], ensemble_version, use_orig_duration=True, save=True)
    start_version += 1


Creating ensemble012:
['model007', 'model009', 'model011', 'model015']


Evaluating the ensemble:   0%|          | 0/488 [00:00<?, ?it/s]

Computing standard results:


Evaluating the ensemble: 100%|██████████| 488/488 [01:54<00:00,  4.34it/s]
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision i

Computing weak from strong results:


Evaluating the ensemble: 100%|██████████| 488/488 [01:52<00:00,  4.31it/s]


## Create final set of ensembles using all good models

In [6]:
from itertools import combinations

In [7]:
models = (['model007', 'model009', 'model011', 'model013', 'model015', 'model024', 'model026bm'])

In [8]:
# All combos of 2
combos = [c for c in combinations(models, 2)]
combos

[('model007', 'model009'),
 ('model007', 'model011'),
 ('model007', 'model013'),
 ('model007', 'model015'),
 ('model007', 'model024'),
 ('model007', 'model026bm'),
 ('model009', 'model011'),
 ('model009', 'model013'),
 ('model009', 'model015'),
 ('model009', 'model024'),
 ('model009', 'model026bm'),
 ('model011', 'model013'),
 ('model011', 'model015'),
 ('model011', 'model024'),
 ('model011', 'model026bm'),
 ('model013', 'model015'),
 ('model013', 'model024'),
 ('model013', 'model026bm'),
 ('model015', 'model024'),
 ('model015', 'model026bm'),
 ('model024', 'model026bm')]

In [9]:
len(combos)

21

In [ ]:
start_version = 20

for i in range(21):

    if start_version < 10:
        ensemble_version = 'ensemble00{}'.format(start_version)
    else:
        ensemble_version = 'ensemble0{}'.format(start_version)
        
    print('\nCreating {}:\n{}'.format(ensemble_version, combos[i]))
    _ = create_ensemble(combos[i], ensemble_version, use_orig_duration=True, save=True)
    start_version += 1


Creating ensemble020:
('model007', 'model009')


Evaluating the ensemble:   0%|          | 0/488 [00:00<?, ?it/s]

Computing results:


Evaluating the ensemble: 100%|██████████| 488/488 [01:27<00:00,  5.24it/s]
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision i


Creating ensemble021:
('model007', 'model011')


...

Evaluating the ensemble: 100%|██████████| 488/488 [01:41<00:00,  4.87it/s]



Creating ensemble037:
('model013', 'model026bm')


Evaluating the ensemble:   0%|          | 0/488 [00:00<?, ?it/s]

Computing results:


Evaluating the ensemble:  79%|███████▊  | 384/488 [01:25<01:15,  1.38it/s]

In [10]:
start_version = 37

for i in range(17, 21):

    if start_version < 10:
        ensemble_version = 'ensemble00{}'.format(start_version)
    else:
        ensemble_version = 'ensemble0{}'.format(start_version)
        
    print('\nCreating {}:\n{}'.format(ensemble_version, combos[i]))
    _ = create_ensemble(combos[i], ensemble_version, use_orig_duration=True, save=True)
    start_version += 1


Creating ensemble037:
('model013', 'model026bm')


Evaluating the ensemble:   0%|          | 0/488 [00:00<?, ?it/s]

Computing results:


Evaluating the ensemble: 100%|██████████| 488/488 [01:32<00:00,  6.18it/s]
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision i


Creating ensemble038:
('model015', 'model024')


...

Evaluating the ensemble: 100%|██████████| 488/488 [01:42<00:00,  4.39it/s]



Creating ensemble040:
('model024', 'model026bm')


Evaluating the ensemble:   0%|          | 0/488 [00:00<?, ?it/s]

Computing results:


Evaluating the ensemble: 100%|██████████| 488/488 [01:36<00:00,  6.19it/s]
